In [1]:
!pip install torchdata
#Restart Kernel

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 47 kB 2.9 MB/s eta 0:00:011


In [1]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 6.2 MB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.1 MB 33.4 MB/s eta 0:00:01
     |████████████████████████████████| 9.9 MB 19.9 MB/s eta 0:00:01
     |████████████████████████████████| 660 kB 37.4 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 2.0 MB/s  eta 0:00:01
     |████████████████████████████████| 457 kB 41.1 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 46.4 MB/s eta 0:00:01
     |████████████████████████████████| 125 kB 33.2 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 7.8 MB/s  eta 0:00:01
ERROR: pydantic 1.8.2 has requirement typing-extensions>=3.7.4.3, but you'll have typing-extensions 3.7.4.1 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you pref

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 14.6 MB 5.4 MB/s eta 0:00:01


✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim


from torchtext.datasets import Multi30k
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab, build_vocab_from_iterator

from torch.utils.data import DataLoader

from collections import Counter


import numpy as np

import random
import math
import time

In [3]:

HID_DIM = 512
N_LAYERS = 4
ENC_DROPOUT = 0.3
DEC_DROPOUT = 0.3
EMB_DIM = 100


In [4]:
import io
from torchtext.utils import download_from_url, extract_archive

url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

de_tokenizer = get_tokenizer('spacy', language='de')
en_tokenizer = get_tokenizer('spacy', language='en')



/opt/conda/lib/python3.7/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')
/opt/conda/lib/python3.7/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [5]:
en_tokenizer("How you doing?")

['How', 'you', 'doing', '?']

In [6]:
from torchtext.vocab import vocab

words = []

def build_vocab(filepath, tokenizer):
    word_lang = []
    count = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            word_lang.append(string_)
            for t in tokenizer(string_):
                count.update([t])
                #print(t)
    words.append(word_lang)
    return  vocab(count, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

In [7]:
print(words[1][1])
print(words[0][1])

Several men in hard hats are operating a giant pulley system.

Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.



In [8]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)

In [9]:
print(en_vocab['the'])
en_vocab.vocab.get_itos()[0]

43


'<unk>'

In [10]:
en_vocab.set_default_index(en_vocab['<unk>'])
de_vocab.set_default_index(de_vocab['<unk>'])

def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    data = []
    for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
        de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                                dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                                dtype=torch.long)
        data.append((de_tensor_, en_tensor_))
    return data

train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [11]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
    de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    return de_batch, en_batch
import gzip
from nltk.stem.porter import *
from nltk.stem import *
from nltk.tokenize import RegexpTokenizer
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [12]:
import spacy
en_nlp = spacy.load("en_core_web_sm")
en_embed_weights = torch.FloatTensor(en_nlp.vocab.vectors)

In [13]:
print(len(en_embed_weights))


0


In [ ]:
!wget https://dumps.wikimedia.org/dewiki/latest/dewiki-latest-pages-articles.xml.bz2

In [42]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, dropout):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.embed = nn.Embedding(vocab_size, embed_dim)
        #self.embed = self.embed.from_pretrained(en_embed_weights, freeze=False)
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, dropout=dropout)
    
    def forward(self, x):
        embedding = self.dropout(self.embed(x))
        outputs, (hidden, cell) = self.lstm(embedding)
        return hidden, cell
    
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_layers, dropout):
        super(Decoder, self).__init__()
        self.vocab_size = vocab_size
        self.dropout = nn.Dropout(dropout)
        self.embed = nn.Embedding(vocab_size, embed_dim)
        
        self.lstm = nn.LSTM(embed_dim, hidden_size, num_layers, dropout=dropout)
        
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, hidden, cell):
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embed(x))
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
        outputs = outputs.squeeze(0)
        predictions = self.fc(outputs)
        return predictions, hidden, cell

In [56]:
import random

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
    
    def forward(self, source, target, teacher_force_ratio=1):
        seq_len = target.size(0)
        batch_size = target.size(1)
        #print("batch_size",batch_size)
        #print("seq_len", seq_len)
        outputs = torch.zeros(
            seq_len, batch_size,   self.decoder.vocab_size
        ).to(self.device)
        
        hidden, cell = self.encoder(source)
        x = target[0,:]
        
        for t in range(1, seq_len):
            predictions, hidden, cell = self.decoder(x, hidden, cell)
            outputs[t] = predictions
            teacher_force = random.random() < teacher_force_ratio
            if teacher_force:
                x = target[t]
            else:
                x = predictions.argmax(1)
                #print("predictions.argmax", predictions.argmax(1))
                
        return outputs

In [72]:


encoder = Encoder(INPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(encoder, decoder, device).to(device)

def init_weights(model):
    for name, param in model.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0007)

In [73]:
model.load_state_dict(torch.load('saved_model.pt'))

<All keys matched successfully>

In [74]:
#from workspace_utils import active_session
epochs = 2
def train():
    model.train()
    if True: #with active_session():
        for epoch in range(epochs):
            #h = torch.zeros(batch_size, 1, encoder_hidden_size), torch.zeros(1, 1, encoder_hidden_size)
            loss_avg = 0
            print("Epoch: ", epoch)
            for src, trg in train_iter:
                
                src = src.to(device)
                trg = trg.to(device)
                model.zero_grad()
                
                output = model(src, trg, teacher_force_ratio=0.1)

                output = output[1:].view(-1, output.size(2))
                trg = trg[1:].reshape(-1)
                
                loss = loss_function(output, trg)
                loss.backward()
                #nn.utils.clip_grad_norm_(model.parameters(), 1)
                optimizer.step()
                loss_avg += loss
                
                torch.save(model.state_dict(), 'saved_model.pt')

            for src, trg in valid_iter:
                
                src = src.to(device)
                trg = trg.to(device)
                model.zero_grad()
                
                output = model(src, trg)

                output = output[1:].view(-1, output.size(2))
                trg = trg[1:].reshape(-1)
                
                loss = loss_function(output, trg)
                loss.backward()
                #nn.utils.clip_grad_norm_(model.parameters(), 1)
                optimizer.step()
                loss_avg += loss
                
                torch.save(model.state_dict(), 'saved_model.pt')
    
                
            print("Epoch Loss: ", loss_avg/len(train_iter))
            loss_avg = 0
            
train()

Epoch:  0
Epoch Loss:  tensor(1.0649, device='cuda:0', grad_fn=<DivBackward0>)
Epoch:  1
Epoch Loss:  tensor(1.0161, device='cuda:0', grad_fn=<DivBackward0>)


In [75]:
model.load_state_dict(torch.load('saved_model.pt'))

<All keys matched successfully>

In [76]:
def evaluate():
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        i =0 
        for source, target in test_iter:
            source = source.to(device)
            target = target.to(device)
            output = model(source, target, teacher_force_ratio=0)
            target = target[1:].reshape(-1)
            output = output[1:].reshape(-1, output.size(2))
            loss = loss_function(output, target)
            epoch_loss += loss.item()
            i = i + 1
    model.train()
    return epoch_loss / (i + 1)

print(evaluate())

2.104138215382894


In [77]:
def preprocess_text(text):
    tokens = de_tokenizer(text)
    tokens.append('<eos>')
    tokens.insert(0,'<bos>')
    return tokens

In [78]:
model.eval()
#print(en_vocab['the'])
en_vocab.vocab.get_itos()[0]
def generate_response(sentence):
    
    sentence = preprocess_text(sentence)
    sentence_index = [en_vocab[word] for word in sentence]
    in_tensor = torch.tensor(sentence_index, dtype=torch.long).view(-1, 1)

    in_tensor = in_tensor.to(device)

    result = model.forward(in_tensor, in_tensor, teacher_force_ratio=0)
    #print(result)
    response = []
    for x in result:
        response.append(en_vocab.vocab.get_itos()[x.argmax()])
    
    return response


def chat():
    
    while(True):
       # try:
            
            sentence = input("You: ")
            if sentence == 'exit':
                break
            else:
                response = generate_response(sentence)
                print("Bot: ", ' '.join(response) )
                
        # except Exception as e:
          #  print(e)
          #  print("Error: Unknown vocabulary word.")
chat()

You: Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem
Bot:  <unk> <unk> - old old into connection . 

You: exit


In [ ]:
"Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem."